In [1]:
"""
CSV - comma separted values; spreadsheet easy to process with code
"""

# Load Data from CSVs as dictionaries: -----------------------------------------------------------------------------------------


import unicodecsv

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

# declaring stuffs
enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')


# Fixing Data Types: ------------------------------------------------------------------------------------------------------------


# look in starter codee...lmao

from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]


# Find the total number of rows and unique IDs: ---------------------------------------------------------------------------------


# find number or rows
print(len(enrollments))

# make a set so only unique values are added to it
def get_unique_students(data):
    unique_students = set()
    for i in data:
        unique_students.add(i['account_key'])
    return unique_students

len(enrollments)
unique_enrolled_students = get_unique_students(enrollments)
print(len(unique_enrolled_students))

len(daily_engagement)
unique_engagement_students = get_unique_students(daily_engagement)
print(len(unique_engagement_students))

len(project_submissions)
unique_project_submitters = get_unique_students(project_submissions)
print(len(unique_project_submitters))


# Find why there are less unique users than total users: ------------------------------------------------------------------------


# loop through enrollments and print accounts that dont have any data for engagements
for i in enrollments:
    student = i['account_key']
    if student not in unique_engagement_students:
        print(i)
        break # delete break to show all

# print accounts that werent canceled on the same day enrolled
num_problem_students = 0
for i in enrollments:
    student = i['account_key']
    if (student not in unique_engagement_students and 
            i['join_date'] != i['cancel_date']):
        print(i)
        num_problem_students += 1

print(num_problem_students)


# Remove data from data set: ----------------------------------------------------------------------------------------------------


# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for i in enrollments:
    if i['is_udacity']:
        udacity_test_accounts.add(i['account_key'])
print(len(udacity_test_accounts))

# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for i in data:
        if i['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(i)
    return non_udacity_data

# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))


# Create new dicitonary from existing data set: ---------------------------------------------------------------------------------


paid_students = {}

for i in non_udacity_enrollments:
    if not i['is_canceled'] or i['days_to_cancel'] > 7:
        account_key = i['account_key']
        enrollment_date = i['join_date']
        paid_students[account_key] = enrollment_date
        
        # changes enrollment date for certain students: accounts for having an account that has a second enroll date
        if account_key not in paid_students or enrollment_date > paid_students[account_key]:
            paid_students[account_key] = enrollment_date

print(len(paid_students))


# idk: ------------------------------------------------------------------------------


# Takes a student's join date and the date of a specific engagement record, and returns True if that engagement record
# happened within one week of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)

print(len(paid_enrollments))
print(len(paid_engagement))
print(len(paid_submissions))

paid_engagement_in_first_week = []
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']

    if within_one_week(join_date, engagement_record_date):
         paid_engagement_in_first_week.append(engagement_record)

print(len(paid_engagement_in_first_week))

1640
1302
1237
743
OrderedDict([('account_key', '1219'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 12, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 12, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)])
OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1101'), ('status', 'current'), ('join_date', datetime.datetime(2015, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_udacity', True), ('is_canceled', False)])
3
6
1622
135656
3634
995
1293
134549
3618


TypeError: unsupported operand type(s) for -: 'str' and 'datetime.datetime'